# Статистика карточек товаров за период


Link: https://dev.wildberries.ru/openapi/analytics#tag/Voronka-prodazh/paths/~1api~1v2~1nm-report~1detail/post


Описание метода
Method: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail

Метод формирует отчёт о товарах, сравнивая ключевые показатели — например, добавления в корзину, заказы и переходы в карточку товара — за текущий период с аналогичным прошлым.


Параметры brandNames,objectIDs, tagIDs, nmIDs могут быть пустыми [], тогда в ответе возвращаются все карточки продавца.


Если выбрано несколько параметров, в ответе будут карточки, в которых есть одновременно все эти параметры. Если карточки не подходят по параметрам запроса, вернётся пустой ответ [].


Можно получить отчёт максимум за последние 365 дней.


В данных предыдущего периода:

Данные в previousPeriod указаны за такой же период, что и в selectedPeriod.
Если дата начала previousPeriod раньше, чем год назад от текущей даты, она будет приведена к виду: previousPeriod.begin = текущая дата - 365 дней.

Максимум 3 запроса в минуту на один аккаунт продавца

Response Schema: application/json

- data	 / object
    - page	 / integer <int32> / Страница
    - isNextPage	 / boolean / Есть ли следующая страница (false - нет, true - есть)
    - cards	 / Array of objects / Array 
        - nmID	 / integer <int32> / Артикул WB
        - vendorCode	 / string / Артикул продавца
        - brandName	 / string / Название бренд
        - tags	 / Array of objects / Ярлыки / Array 
            - id	 / integer <int32> / ID ярлыка
            - name	 / string / Название ярлыка
        - object	 / object / Предмет
            - id	 / integer <int32> / ID предмета
            - name	/ string/ Название предмета/ 
        - statistics	/ object/ Статистика/ 
            - selectedPeriod	/ object/ Запрашиваемый период/ 
            - begin	/ string/ Начало периода
            - end	/ string/ Конец периода/ 
            - openCardCount	/ integer <int32>/ Количество переходов в карточку товара
            - addToCartCount	/ integer <int32>/ Положили в корзину, штук
            - ordersCount	/ integer <int32>/ Заказали товаров, шт
            - ordersSumRub	/ integer <int32>/ Заказали на сумму, руб.
            - buyoutsCount	/ integer <int32>/ Выкупили товаров, шт.
            - buyoutsSumRub	/ integer <int32>/ Выкупили на сумму, руб.
            - cancelCount	/ integer <int32>/ Отменили товаров, ш
            - cancelSumRub	/ integer <int32>/ Отменили на сумму, руб.
            - avgPriceRub	/ integer <int32>/ Средняя цена, руб.
            - avgOrdersCountPerDay	/ integer <int32>/ Среднее количество заказов в день, шт.
            - conversions	/ object/ Конверсии
            - addToCartPercent	/ integer <int32>/ Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
            - artToOrderPercent	/ integer <int32>/ Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
            - buyoutsPercent	/ integer <int32>/ Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
            - previousPeriod	/ object/ Статистика за предыдущие 30 дней
            - periodComparison	/ object/ Сравнение двух периодов, в процентах
        - stocks	/ object/ Остатки/ 
            - stocksMp	 / integer <int32> / Остатки МП, шт. (Общее количество остатков на складе продавца)
            - stocksWb	 / integer <int32> / Остатки на складах WB (Общее количество остатков на складах WB)
- error	/ boolean/ Флаг ошибки
- errorText	/ string/ Описание ошибки
- additionalErrors / Array of object / Дополнительные ошибки

In [1]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import date, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [2]:
# Retrieve API keys from environment variables
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv('KeySmart')
password = os.getenv('ClickHouse')


# Define headers for each project
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set

}


In [3]:
# Define the API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"

# Set fixed date range for testing
yesterday_start = '2025-05-30 00:00:00'
yesterday_end = '2025-05-30 23:59:59'

def get_report(url, headers, begin, end, page, project_name):
    all_data = []
    max_retries = 3
    retry_delay = 30  # seconds
    request_timeout = 60  # seconds
    
    while True:
        for attempt in range(max_retries):
            try:
                # Define the request body with all required parameters
                request_body = {
                    "period": {
                        "begin": begin,
                        "end": end
                    },
                    "orderBy": {
                        "field": "ordersSumRub",
                        "mode": "desc"
                    },
                    "page": page,
                    "timezone": "Europe/Moscow",  # Added as recommended in API docs
                    "brandNames": [],  # Empty array to get all brands
                    "objectIDs": [],   # Empty array to get all objects
                    "nmIDs": []        # Empty array to get all items
                }

                json_data = json.dumps(request_body, ensure_ascii=False)
                
                # Send the POST request with timeout
                response = requests.post(
                    url, 
                    headers=headers, 
                    data=json_data, 
                    timeout=request_timeout
                )
                
                # Handle rate limiting (429 status code)
                if response.status_code == 429:
                    wait_time = int(response.headers.get('Retry-After', retry_delay))
                    print(f"Rate limit exceeded for {project_name}. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    continue
                
                # Check for successful response
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} for {project_name}, page {page}")
                    print("Response text:", response.text)
                    if attempt == max_retries - 1:
                        return all_data
                    time.sleep(retry_delay)
                    continue
                
                data = response.json()
                
                # Validate response structure
                if not data.get('data') or not isinstance(data['data'].get('cards'), list):
                    print(f"Invalid data structure received for {project_name}, page {page}")
                    return all_data
                
                cards = data['data']['cards']
                
                # Check if we have any data
                if not cards:
                    print(f"No more data available for {project_name}")
                    return all_data
                
                all_data.extend(cards)
                print(f"Page {page} retrieved successfully for {project_name} (got {len(cards)} items)")
                
                # Check if there are more pages (using isNextPage flag)
                if not data['data'].get('isNextPage', False):
                    print(f"Reached last page for {project_name}")
                    return all_data
                
                # Prepare for next page
                page += 1
                time.sleep(5)  # Reduced delay between pages to 5 seconds
                break  # Success, break out of retry loop
                
            except requests.exceptions.RequestException as e:
                print(f"Request failed for {project_name}, page {page}: {str(e)}")
                if attempt == max_retries - 1:
                    return all_data
                time.sleep(retry_delay)
                continue
            except Exception as e:
                print(f"Unexpected error for {project_name}, page {page}: {str(e)}")
                return all_data

# Get data for each project
data_guten = get_report(url, headers_guten, yesterday_start, yesterday_end, 1, 'WB-GutenTech')
data_giper = get_report(url, headers_giper, yesterday_start, yesterday_end, 1, 'WB-ГиперМаркет')
data_kitchen = get_report(url, headers_kitchen, yesterday_start, yesterday_end, 1, 'WB-KitchenAid')
data_smart = get_report(url, headers_smart, yesterday_start, yesterday_end, 1, 'WB-Smart-Market')

Request failed with status code 503 for WB-GutenTech, page 1
Response text: "Service Unavailable"

Request failed with status code 503 for WB-GutenTech, page 1
Response text: "Service Unavailable"

Page 1 retrieved successfully for WB-GutenTech (got 1000 items)
Page 2 retrieved successfully for WB-GutenTech (got 942 items)
Reached last page for WB-GutenTech
Page 1 retrieved successfully for WB-ГиперМаркет (got 1000 items)
Page 2 retrieved successfully for WB-ГиперМаркет (got 65 items)
Reached last page for WB-ГиперМаркет
Page 1 retrieved successfully for WB-KitchenAid (got 97 items)
Reached last page for WB-KitchenAid
Page 1 retrieved successfully for WB-Smart-Market (got 96 items)
Reached last page for WB-Smart-Market


In [4]:
# Function to flatten the JSON data for the current period
def flatten_json_current_period(cards):
    flattened_data = []
    for card in cards:
        nmID = card["nmID"]
        vendorCode = card["vendorCode"]
        brandName = card["brandName"]
        objectID = card["object"]["id"]
        objectName = card["object"]["name"]
        

        # Extract statistics for the selected period
        selected_period = card["statistics"]["selectedPeriod"]

        flattened_data.append({
            "nmID": nmID,
            "vendorCode": vendorCode,
            "brandName": brandName,
            "objectID": objectID,
            "objectName": objectName,
            "begin": selected_period["begin"],
            "end": selected_period["end"],
            "openCardCount": selected_period["openCardCount"],
            "addToCartCount": selected_period["addToCartCount"],
            "ordersCount": selected_period["ordersCount"],
            "ordersSumRub": selected_period["ordersSumRub"],
            "buyoutsCount": selected_period["buyoutsCount"],
            "buyoutsSumRub": selected_period["buyoutsSumRub"],
            "cancelCount": selected_period["cancelCount"],
            "cancelSumRub": selected_period["cancelSumRub"],
            "avgOrdersCountPerDay": selected_period["avgOrdersCountPerDay"],
            "avgPriceRub": selected_period["avgPriceRub"],
            "addToCartPercent": selected_period["conversions"]["addToCartPercent"],
            "cartToOrderPercent": selected_period["conversions"]["cartToOrderPercent"],
            "buyoutsPercent": selected_period["conversions"]["buyoutsPercent"],
            "stocksMp": card["stocks"]["stocksMp"],
            "stocksWb": card["stocks"]["stocksWb"]
        })

    return flattened_data

# Convert the flattened data to a DataFrame
flattened_data_guten = flatten_json_current_period(data_guten)
flattened_data_giper = flatten_json_current_period(data_giper)
flattened_data_kitchen = flatten_json_current_period(data_kitchen)
flattened_data_smart = flatten_json_current_period(data_smart)

df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)

# Add the 'Project' column to each DataFrame before concatenation
df_guten['Project'] = 'WB-GutenTech'
df_giper['Project'] = 'WB-ГиперМаркет'
df_kitchen['Project'] = 'WB-KitchenAid'
df_smart['Project'] = 'WB-Smart-Market'

# Combine all campaign data
combined_df = pd.concat([df_guten, df_giper, df_kitchen, df_smart], ignore_index=True)
combined_df['Marketplace'] = 'Wildberries'
print("Columns in combined_campaigns:", combined_df.columns.tolist())

# Display the DataFrame
combined_df

Columns in combined_campaigns: ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'end', 'openCardCount', 'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 'cancelSumRub', 'avgOrdersCountPerDay', 'avgPriceRub', 'addToCartPercent', 'cartToOrderPercent', 'buyoutsPercent', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']


,nmID,vendorCode,brandName,objectID,objectName,begin,end,openCardCount,addToCartCount,ordersCount,...,cancelSumRub,avgOrdersCountPerDay,avgPriceRub,addToCartPercent,cartToOrderPercent,buyoutsPercent,stocksMp,stocksWb,Project,Marketplace
0,214664991,SI9682DB,Braun,643,Утюги,2025-05-30 00:00:00,2025-05-30 23:59:59,92,8,5,...,0,5,11530,9,63,100,190,0,WB-GutenTech,Wildberries
1,159488075,I01475,AND,594,Массажеры электрические,2025-05-30 00:00:00,2025-05-30 23:59:59,377,41,10,...,0,10,4563,11,24,100,0,319,WB-GutenTech,Wildberries
2,25624340,Б0027735,Braun,620,Бритвы электрические,2025-05-30 00:00:00,2025-05-30 23:59:59,113,11,5,...,0,5,6540,10,45,100,0,2,WB-GutenTech,Wildberries
3,147720619,61-R1200s,Braun,620,Бритвы электрические,2025-05-30 00:00:00,2025-05-30 23:59:59,52,4,2,...,0,2,16290,8,50,100,0,0,WB-GutenTech,Wildberries
4,267706794,91887,KitchenAid,616,Чайники электрические,2025-05-30 00:00:00,2025-05-30 23:59:59,16,1,1,...,0,1,32229,6,100,100,0,0,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,288465482,8700216589031,GILLETTE,1527,Бритвы безопасные,2025-05-30 00:00:00,2025-05-30 23:59:59,9,0,0,...,0,0,0,0,0,0,0,8,WB-Smart-Market,Wildberries
3196,288465484,8700216589017,GILLETTE,1527,Бритвы безопасные,2025-05-30 00:00:00,2025-05-30 23:59:59,13,0,0,...,0,0,0,0,0,0,0,61,WB-Smart-Market,Wildberries
3197,288465485,8700216588713,GILLETTE,1527,Бритвы безопасные,2025-05-30 00:00:00,2025-05-30 23:59:59,14,0,0,...,0,0,0,0,0,0,0,3,WB-Smart-Market,Wildberries
3198,288465487,8700216588911,GILLETTE,1527,Бритвы безопасные,2025-05-30 00:00:00,2025-05-30 23:59:59,9,0,0,...,0,0,0,0,0,0,0,71,WB-Smart-Market,Wildberries


In [5]:
print(combined_df["Project"].unique())

['WB-GutenTech' 'WB-ГиперМаркет' 'WB-KitchenAid' 'WB-Smart-Market']


In [6]:
# Save the DataFrame to an Excel file
excel_file_path = 'report_data.xlsx'
combined_df.to_excel(excel_file_path, index=False)

print(f"Data saved to {excel_file_path} successfully!")

Data saved to report_data.xlsx successfully!


In [7]:
# Keep only the desired columns
columns_to_keep = ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'openCardCount', 'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 'cancelSumRub', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_df[columns_to_keep].copy()
filtered_df['begin'] = pd.to_datetime(filtered_df['begin'])
filtered_df
df_final=filtered_df.copy()

## Inserting the data

In [8]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [9]:
# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(filtered_df.dtypes)

# Ensure the DataFrame has the correct columns
columns = ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'openCardCount', 
           'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 
           'cancelSumRub', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']

# Reorder columns to match the expected order
data_organized = filtered_df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in data_organized.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'order_history_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmID                       int64
vendorCode                object
brandName                 object
objectID                   int64
objectName                object
begin             datetime64[ns]
openCardCount              int64
addToCartCount             int64
ordersCount                int64
ordersSumRub               int64
buyoutsCount               int64
buyoutsSumRub              int64
cancelCount                int64
cancelSumRub               int64
stocksMp                   int64
stocksWb                   int64
Project                   object
Marketplace               object
dtype: object
Sample data to insert: [(214664991, 'SI9682DB', 'Braun', 643, 'Утюги', Timestamp('2025-05-30 00:00:00'), 92, 8, 5, 57650, 2, 23060, 0, 0, 190, 0, 'WB-GutenTech', 'Wildberries'), (159488075, 'I01475', 'AND', 594, 'Массажеры электрические', Timestamp('2025-05-30 00:00:00'), 377, 41, 10, 45630, 8, 36540, 0, 0, 0, 319, 'WB-GutenTech', 'Wildberries'), (25624340, 'Б00277